# Urban Flood Management and Emergency Response Use Case

## Scenario

As urban areas face increasing challenges due to climate change and rapid development, managing flood risks has become crucial for ensuring public safety and minimizing economic damage. This use case focuses on using data-driven approaches to predict flood zones, optimize emergency response, and inform infrastructure upgrades for long-term resilience.

By leveraging datasets related to stormwater pits, drain pipes, and geospatial information, this use case aims to simulate water flow during heavy rainfall, identify vulnerable areas, and enhance response times for emergency teams. Urban planners and engineers can use the insights from this analysis to prioritize maintenance efforts, upgrade outdated infrastructure, and mitigate future flooding risks.

**At the end of this use case, you will:**
- Learn how to clean and integrate multiple datasets using Pandas and GIS tools.
- Perform exploratory data analysis to simulate water flow and predict flood zones.
- Use data visualization techniques to map high-risk areas for flooding.
- Generate actionable insights to inform emergency response planning and urban development strategies.

---

## Project Stages

### 1. Data Collection:
- Gather datasets such as stormwater pits and drain pipes. These include spatial, material, and condition information necessary for simulating water flow.
- Use geospatial tools to extract and process location data (latitude and longitude) from both datasets.

### 2. Data Cleaning and Preparation:
- Load the datasets into data analysis tools like Pandas.
- Inspect and clean the data to handle any missing or incorrect values.
- Merge the datasets based on common fields like location or asset number to create a unified dataset for analysis.

### 3. Data Analysis:
- Analyze drainage speed, pipe dimensions, and overflow capacities to simulate potential flood risks in different parts of the city.
- Combine this with GIS mapping to visually represent flood-prone zones.
- Assess the condition and material of stormwater pits and drain pipes to identify infrastructure weaknesses.

### 4. Determine High-Risk Areas:
- Identify areas that are likely to experience flooding based on the flow simulation.
- Highlight regions with older or poorly maintained infrastructure that might require urgent upgrades.

### 5. Visualization:
- Create visualizations using GIS tools to map high-risk flood zones and identify key areas for emergency response.
- Present the findings in a clear and impactful manner to support urban planning decisions.

---

## Expected Outcome

The expected outcome of this project is to develop a comprehensive flood risk map that urban planners can use to prepare for future flooding events. Emergency response teams will be able to optimize routes and prioritize rescue efforts, while infrastructure maintenance crews can focus on vulnerable areas to prevent floods before they occur. These insights will lead to a safer and more resilient urban environment.

# Package/Library Imports

In [1]:
from config import API_KEY # import API_KEY from config file

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests 
import pandas as pd 
from io import StringIO
import squarify

# Read data using API

In [8]:
# **Preferred Method**: Export Endpoint

#Function to collect data 
def collect_data(dataset_id):
    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    dataset_id = dataset_id
    format = 'csv'

    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
    'select': '*',
    'limit': -1, # all records
    'lang': 'en',
    'timezone': 'UTC',
    'api_key': API_KEY #use if use datasets require API key permissions
    }

    # GET request
    response = requests.get(url, params=params)
    if response.status_code == 200:
        # StringIO to read the CSV data
        url_content = response.content.decode('utf-8')
        dataset = pd.read_csv(StringIO(url_content), delimiter=';')
        return dataset 
    else:
        print(f'Request failed with status code {response.status_code}')

# Read data using the function
stormwater_pits = collect_data('stormwater-pits')
drainpipes = collect_data('drainpipes')
melb_photo_mesh = collect_data('city-of-melbourne-3d-textured-mesh-photomesh-2020')

# Learn and explore the data

## Stormwater Pits

In [4]:
print(f'Shape of the stormwater pits:\t{stormwater_pits.shape}')
stormwater_pits.head(5)

Shape of the stormwater pits:	(21113, 13)


,asset_number,asset_description,construction_material_lupvalue,grate_length,grate_material_lupvalue,grate_width,model_descr_lupvalue,model_no_lupvalue,object_type_lupvalue,overflow_kerb_lupvalue,lat,lon,location
0,1596911,SWD Pit - Grated Manhole,Brick,NaN,Cast Iron,NaN,SWD Pit - Grated Manhole,300413-001,Grated Manhole,NaN,-37.792906,144.956536,"-37.79290628271831, 144.95653615629308"
1,1759114,SWD Pit - Lane Type,In Situ Concrete,450.0,Mild Steel,450.0,SWD Pit - Laneway Type - Unclassified,300407-000,Lane Type,NaN,-37.813643,144.967603,"-37.81364319503544, 144.96760262284465"
2,1150456,SWD Pit - System Node,Brick,0.0,NaN,0.0,SWD Pit - System Node,300414-000,System Node,NaN,-37.815175,144.963504,"-37.81517520847289, 144.96350385914948"
3,1146866,SWD Pit - Lane Type,In Situ Concrete,550.0,Mild Steel,550.0,SWD Pit - Laneway Type - Non Standard Frame an...,300407-003,Lane Type,NaN,-37.793212,144.956170,"-37.79321210088348, 144.95617023562534"
4,1146782,SWD Pit - Grated Side Entry,Brick,900.0,Mild Steel,430.0,SWD Pit - Grated Side Entry Pit - Non Standard...,300402-003,Grated Side Entry,NaN,-37.793758,144.955482,"-37.79375834585664, 144.95548209270686"


In [5]:
print('-'*20,'Stormwater Pits Info','-'*20)
print()
# display all columns, non-null count and their data types
print(stormwater_pits.info())

-------------------- Stormwater Pits Info --------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21113 entries, 0 to 21112
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   asset_number                    21113 non-null  int64  
 1   asset_description               21113 non-null  object 
 2   construction_material_lupvalue  20744 non-null  object 
 3   grate_length                    9445 non-null   float64
 4   grate_material_lupvalue         10154 non-null  object 
 5   grate_width                     9440 non-null   float64
 6   model_descr_lupvalue            20691 non-null  object 
 7   model_no_lupvalue               19704 non-null  object 
 8   object_type_lupvalue            21066 non-null  object 
 9   overflow_kerb_lupvalue          5286 non-null   object 
 10  lat                             21113 non-null  float64
 11  lon                          

## Drainpipes data

In [6]:
print(f'Shape of the Drainepipes data:\t{drainpipes.shape}')
drainpipes.head(5)

Shape of the Drainepipes data:	(17242, 17)


,geo_point_2d,geo_shape,built,dnstr_pit,upstr_inv,upstr_pit,ref,width_mm,condition,carrying,operator,diameter,material,height_mm,form,mat_desc,dnstr_inv
0,"-37.832359984743945, 144.9723773392074","{""coordinates"": [[144.972362288225, -37.832355...",NaN,1472763.0,NaN,1472761.0,1473265.0,NaN,3.000000,Stormwater,City of Melbourne,150.0,Unknown,NaN,Pipe,NaN,NaN
1,"-37.81302804684621, 144.93898090981813","{""coordinates"": [[144.93891931812107, -37.8129...",2006-01-01,NaN,NaN,NaN,1379195.0,0.0,4.795374,Stormwater,City of Melbourne,450.0,Concrete,0.0,Pipe,NaN,NaN
2,"-37.82026540119676, 144.9575218117356","{""coordinates"": [[144.95753958893778, -37.8202...",1942-01-01,1152853.0,NaN,1152688.0,1138399.0,0.0,3.271437,Stormwater,City of Melbourne,750.0,Brick,0.0,Pipe,Brick Barrel,NaN
3,"-37.830759012745204, 144.9741777544205","{""coordinates"": [[144.97422754662665, -37.8306...",NaN,1472766.0,NaN,1472784.0,1473274.0,NaN,3.000000,Stormwater,City of Melbourne,150.0,Plastic,NaN,Pipe,NaN,NaN
4,"-37.83103639062876, 144.97380384007778","{""coordinates"": [[144.9738022883896, -37.83102...",NaN,1472768.0,NaN,1472810.0,1473277.0,NaN,3.000000,Stormwater,City of Melbourne,150.0,Plastic,NaN,Pipe,NaN,NaN


In [7]:
print('-'*20,'Drainpipes Info','-'*20)
print()
# display all columns, non-null count and their data types
print(drainpipes.info())

-------------------- Drainpipes Info --------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17242 entries, 0 to 17241
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   geo_point_2d  17242 non-null  object 
 1   geo_shape     17242 non-null  object 
 2   built         15634 non-null  object 
 3   dnstr_pit     14305 non-null  float64
 4   upstr_inv     1012 non-null   float64
 5   upstr_pit     14224 non-null  float64
 6   ref           17157 non-null  float64
 7   width_mm      16136 non-null  float64
 8   condition     15812 non-null  float64
 9   carrying      17242 non-null  object 
 10  operator      17155 non-null  object 
 11  diameter      17079 non-null  float64
 12  material      17242 non-null  object 
 13  height_mm     16136 non-null  float64
 14  form          17242 non-null  object 
 15  mat_desc      13020 non-null  object 
 16  dnstr_inv     920 non-null    float64
dtypes: float64

## Melbourne 3D Photomesh

In [9]:
print(f'Shape of the Melb 3d photomesh data:\t{melb_photo_mesh.shape}')
melb_photo_mesh.head(5)

Shape of the Melb 3d photomesh data:	(0, 8)


,recordid,_record_id,record_timestamp,_record_timestamp,record_size,_record_size,resource_id,_resource_id
